# ASTR597 -- Final Project (Import PTF file into LSD)
Kolby Weisenburger

3 March 2016

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
from astropy.table import Table
import pandas as pd
import os 
%matplotlib inline

In [2]:
import os
os.environ['NWORKERS'] = '8'
os.environ['LSD_CACHEDIR'] = os.environ['HOME'] + '/tempdir'
os.environ['LSD_TEMPDIR'] = os.environ['HOME'] + '/tempdir'
os.environ['LSD_DB'] = os.environ['HOME']+'/lsddb:/astro/apps6/opt/lsddb'

In [3]:
from lsd import DB
from lsd.bounds import beam, rectangle

In [52]:
!  cat ptf.yaml

# Schema for PTF catalog
filters: {complevel: 5, complib: blosc}
schema:
  common:
    primary_key: ra
    spatial_keys: [ra, dec]
    columns:
    - [ra, f8] 
    - [dec, f8] 
    - [np, i4]
    - [nt, i4]
    - [ptffield, i8]                                                                                                                
    - [ci, i4]                                                                                                                      
    - [flag, i4]                                                                                                                    
    - [bstrms, f8]                                                                                                                  
    - [rptf, f8]                                                                                                                    
    - [rerr, f8]                                                                                                                

In [51]:
! lsd-admin create table --schema ptf.yaml ptf --drop-existing

Table 'ptf' dropped.

-------- committing 20160321215242.159157 [ptf] ---------
[ptf] Updating tablet catalog: [256 el.]::::::::::::::::::::>  0.25 sec
[ptf] Updating neighbors: Already up to date.
[ptf] Updating tablet catalog: [256 el.]::::::::::::::::::::>  0.25 sec
[ptf] Updating stats: [0 el.]>  0.00 sec
[ptf] Marking tablets read-only...
----------- success 20160321215242.159157 [ptf] ---------

Table 'ptf' created.


In [44]:
# learn how to import a text file
! lsd-import text -h 

usage: lsd-import text [-h] [-d DELIMITER] [-c COLS] [--cols-file COLS_FILE]
                       [-f] [--import-primary-key] [--skip-header SKIP_HEADER]
                       [--dms DMS] [--hms HMS] [--set SET]
                       table file [file ...]

positional arguments:
  table                 Name of the table into which to import
  file                  One or more text files. If ending in .gz or .bz2, they
                        are assumed to be compressed.

optional arguments:
  -h, --help            show this help message and exit
  -d DELIMITER, --delimiter DELIMITER
                        The string used to separate values. By default, any
                        consecutive whitespaces will act as delimiter
  -c COLS, --cols COLS  Comma separated list of <colname>:<file_column>, where
                        <file_column> is a 1-based index of the column in the
                        file that is to be loaded into <colname>
  --cols-file COLS_FILE
              

In [56]:
! cat ptf.map

ra 1
dec 2
np 3
nt 4
bstrms 5
rptf 6
rerr 7
rmerr 8
rperr 9
mutype 10
errmu 11
ptffield 12
ci 13
flag 14


In [62]:
! lsd-import text --cols-file ptf.map ptf ptf_catalog.txt

Importing from 1 pieces:
------ rolling back 20160321215711.339247 ---------

Traceback (most recent call last):
  File "/astro/apps6/opt/anaconda2.4/bin/lsd-import", line 88, in <module>
    args.func(args)
  File "/astro/apps6/opt/anaconda2.4/bin/lsd-import", line 53, in chunk_importer
    import_from_chunks(db, importer, chunks)
  File "/astro/apps6/opt/anaconda2.4/bin/lsd-import", line 30, in import_from_chunks
    for (chunk, nloaded, nin) in pool.imap_unordered(chunks, import_from_chunks_aux, (db, importer,), progress_callback=pool2.progress_pass):
  File "/astro/apps6/opt/anaconda2.4/lib/python2.7/site-packages/lsd/pool2.py", line 466, in imap_unordered
    for result in mapper(item, *mapper_args):
  File "/astro/apps6/opt/anaconda2.4/bin/lsd-import", line 20, in import_from_chunks_aux
    ret = importer(db, chunk, *importer_args)
  File "/astro/apps6/opt/anaconda2.4/lib/python2.7/site-packages/lsd/importers/text.py", line 79, in __call__
    ids = db.table(self.tabname).append(